# 1 Image part
- pysekiro
    - img_tools
        - \__init__.py
        - adjustment.py
        - get_status.py
        - get_vertices.py
        - grab_screen.py

## 1.1 grab screen

[grab_screen.py](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/img_tools/grab_screen.py)  
source code: [Sentdex/pygta5/blob/master/grabscreen.py](https://github.com/Sentdex/pygta5/blob/master/grabscreen.py)

In [ ]:
# 运行以下代码，就会抓取当前的屏幕图像
import cv2
from pysekiro.img_tools.grab_screen import get_screen

screen = get_screen()
cv2.imshow('screen', screen)
cv2.waitKey(0)

print('shape', screen.shape)

## 1.2 get vertices

[get_vertices.py](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/img_tools/get_vertices.py)

![demo.gif](https://github.com/ricagj/pysekiro/blob/main/imgs/demo.gif?raw=true)

As shown in the above demonstration(demo.gif), in the pop-up window, use the left mouse button to click the **lower left**, **upper left**, **upper right**, and **lower right** in sequence, a total of 4 times, and then press the "ESC" key on the keyboard, it will automatically return to x, x_w, y, y_h.
(Note: The order of the clicks is stipulated, and the number of clicks is also stipulated)

In [ ]:
import cv2

from pysekiro.img_tools.get_vertices import get_xywh
from pysekiro.img_tools.grab_screen import get_screen

img = get_screen()

get_xywh(img)

## 1.3 get status

[get_status.py](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/img_tools/get_status.py)

![get_status](https://github.com/ricagj/pysekiro/blob/main/imgs/status.jpg?raw=true)

The states to be acquired are: Self_HP, Self_Posture, Target_HP, Target_Posture.
- How to obtain
    1. Get **x, x_w, y, y_h** from [get_vertices.py](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/img_tools/get_vertices.py).
    2. Image processing
    3. Quantify into numbers

### 1.3.1 get HP

1. Extract the ROI
2. The color channel is separated, and the G (Green) channel is taken
3. Image threshold processing
4. Convert to a number

~~~python
def get_Self_HP(img):
    img_roi = roi(img, x=48, x_w=305, y=409, y_h=409+1)

    b, g ,r =cv2.split(img_roi)    # Color channel separation

    retval, img_th = cv2.threshold(g, 50, 255, cv2.THRESH_TOZERO)              # Image threshold processing, if the pixel value is lower than 50, set it to 0
    retval, img_th = cv2.threshold(img_th, 70, 255, cv2.THRESH_TOZERO_INV)    # Image threshold processing, if the pixel value is higher than 70, set it to 0

    target_img = img_th[0]
    if 0 in target_img:
        Self_HP = np.argmin(target_img)
    else:
        Self_HP = len(target_img)

    return Self_HP

def get_Target_HP(img):
    img_roi = roi(img, x=48, x_w=216, y=41, y_h=41+1)

    b, g ,r =cv2.split(img_roi)    # Color channel separation

    retval, img_th = cv2.threshold(g, 25, 255, cv2.THRESH_TOZERO)             # Image threshold processing, if the pixel value is lower than 25, set to 0
    retval, img_th = cv2.threshold(img_th, 70, 255, cv2.THRESH_TOZERO_INV)    # Image threshold processing, if the pixel value is higher than 70, set it to 0

    target_img = img_th[0]
    if 0 in target_img:
        Target_HP = np.argmin(target_img)
    else:
        Target_HP = len(target_img)
    
    return Target_HP
~~~

![demo_get_Self_HP.gif](https://github.com/ricagj/pysekiro/blob/main/imgs/demo_get_Self_HP.gif?raw=true)

### 1.3.2 get Posture

- Only read the right half of the Posture
1. Extract the ROI
2. The color channel is separated, and the R (Red) channel is taken
3. Wait for the middle line (white line) of the posture to appear
    - Use cv2.Canny() to detect edges
4. Convert to a number

~~~python
def get_Self_Posture(img):
    img_roi = roi(img, x=401, x_w=490, y=389, y_h=389+1)
    b, g ,r =cv2.split(img_roi)    # Color channel separation

    white_line = r[0][0]
    if 155 < white_line < 170 or white_line > 250:
        canny = cv2.Canny(cv2.GaussianBlur(r,(3,3),0), 0, 100)
        Self_Posture =  np.argmax(canny)
    else:
        Self_Posture = 0

    if white_line > 250 and Self_Posture < 10:
        Self_Posture == len(canny)

    return Self_Posture

def get_Target_Posture(img):
    img_roi = roi(img, x=401, x_w=553, y=29, y_h=29+1)
    b, g ,r =cv2.split(img_roi)    # Color channel separation

    white_line = r[0][0]
    if white_line > 190:
        canny = cv2.Canny(cv2.GaussianBlur(r,(3,3),0), 0, 100)
        Target_Posture =  np.argmax(canny)
    else:
        Target_Posture = 0

    if white_line > 250 and Target_Posture < 10:
        Target_Posture == len(canny)

    return Target_Posture
~~~

# 2 Key part (Button part)
- pysekiro
    - key_tools
        - \__init__.py
        - actions.py
        - direct_keys.py
        - get_keys.py

## 2.1 get keys

[get_keys.py](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/key_tools/get_keys.py)  
source code: [Sentdex/pygta5/blob/master/getkeys.py](https://github.com/Sentdex/pygta5/blob/master/getkeys.py)

[keys.py](https://github.com/Sentdex/pygta5/blob/master/keys.py) lines 146 to 254
~~~python
# virtual keys
vk = {
    # 'W' : 0x57,
    # 'S' : 0x53,
    # 'A' : 0x41,
    # 'D' : 0x44,
    'LSHIFT' : 0xA0,
    'SPACE'  : 0x20,

    'J' : 0x4A,
    'K' : 0x4B,

    'T' : 0x54,
    'P' : 0x50
}
~~~

Detect the key currently being pressed
~~~python
def key_check():
    keys = []
    for key in ['LSHIFT', 'SPACE', 'J', 'K', 'T', 'P']:    # 'W', 'S', 'A', 'D', 
        if wapi.GetAsyncKeyState(vk[key]):
            keys.append(key)
    return keys
~~~

## 2.2 direct keys

[direct_keys.py](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/key_tools/direct_keys.py)  
source code: [Sentdex/pygta5/blob/master/directkeys.py](https://github.com/Sentdex/pygta5/blob/master/directkeys.py)

PressKey  
ReleaseKey

## 2.3 actions

[actions.py](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/key_tools/actions.py)  
source code: [Sentdex/pygta5/blob/master/3.%20test_model.py](https://github.com/Sentdex/pygta5/blob/master/3.%20test_model.py)

[keys.py](https://github.com/Sentdex/pygta5/blob/master/keys.py) lines 35 to 143
~~~python
# direct keys
dk = {
    'W' : 0x11,
    'S' : 0x1F,
    'A' : 0x1E,
    'D' : 0x20,
    'LSHIFT' : 0x2A,
    'SPACE'  : 0x39,

    'Y'       : 0x15,
    'NUMPAD1' : 0x4F,

    'J' : 0x24,
    'K' : 0x25,
}
~~~

e.g.
~~~python
def Move_Forward():
    PressKey(dk['W'])
    time.sleep(0.01)    # Key duration (control long press and short press)
    ReleaseKey(dk['W'])
~~~

When this function is called, the game will receive the key signal sent by the program, and then drive the operating object to make the corresponding action.